<a href="https://colab.research.google.com/github/luky048-ship-it/My-training-Slubik-Stanislav/blob/main/1_pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание: Pydantic


## Важно!

- При выполнении задания используем точные типы (`EmailStr`, `HttpUrl`, `SecretStr`, `Decimal`, конкретные `Enum`).
- Придерживаемся принципа разделения валидаций: проверка поля — в `field_validator`, сквозные зависимости — в `model_validator`


## Задача 1. Профиль пользователя (валидация полей)

Постройте модель профиля пользователя для внутренней CRM:

**Требования**
1. Обязательные поля: `id: UUID`, `email: EmailStr`, `name: str`.
2. Опциональные поля: `website: HttpUrl | None`, `bio: str | None`.
3. Пароль хранится как `SecretStr`, должен быть не короче 8 символов.
4. Имя (`name`) нормализуйте: тримминг + одна пробельная последовательность между словами + первая буква каждого слова заглавная.
5. Если указан `website`, домен сайта не должен совпадать с доменом `email` (смысл: личный сайт != корпоративная почта).

Подсказки: используйте `field_validator` для нормализации и локальных проверок; и `model_validator(mode="after")` для проверки зависимости `email` ↔ `website`.


In [ ]:
!pip install -U pydantic[email,timezone] -q

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field, EmailStr, HttpUrl, SecretStr
from pydantic import field_validator, model_validator
from uuid import UUID

class UserProfile(BaseModel):
    # TODO: опишите поля согласно требованиям
    ...

    # TODO: нормализация имени
    @field_validator("name")
    @classmethod
    def normalize_name(cls, v: str) -> str:
        ...

    # TODO: проверка длины пароля
    @field_validator("password")
    @classmethod
    def password_strength(cls, v: SecretStr) -> SecretStr:
        ...

    # TODO: сквозная проверка доменов email/website
    @model_validator(mode="after")
    def check_domains(self):
        ...
        return self


## Задача 2. Валидация функции заказа (`@validate_call`)

Реализуйте функцию `place_order`, которая принимает:
- `user_id: UUID`
- `sku: str` (артикул, только заглавные буквы/цифры, длина 3–12)
- `quantity: int` (>0)
- `price: Decimal` (>= 0), округляется банковским методом до 2 знаков

Функция должна возвращать словарь с ключами: `user_id`, `sku`, `quantity`, `price`, `amount` (quantity × price).

Используйте `@validate_call` и локальные проверки через обычный код (или вспомогательные валидаторы `TypeAdapter` не используем).


In [ ]:
from pydantic import validate_call
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
import re

# TODO: реализуйте функцию с @validate_call
@validate_call
def place_order(user_id, sku, quantity, price):
    ...


## Задача 3. Модель заказа с бизнес-правилами

Смоделируйте заказ в магазине цифровых товаров.

**Требования**
- `OrderStatus: Enum` со значениями `new`, `paid`, `delivered`, `canceled`.
- Модель `OrderItem`:
  - `sku: str` как в задаче 2
  - `qty: int` (>0)
  - `unit_price: Decimal` (>=0) округление до 2 знаков
- Модель `Order`:
  - `id: UUID`
  - `user_email: EmailStr`
  - `items: list[OrderItem]` (не пустой)
  - `status: OrderStatus = 'new'`
  - `created_at: datetime` (по умолчанию `datetime.utcnow`)
  - Расчитанное поле `total: Decimal` — сумма по всем позициям
  - В `model_validator(mode="after")` запретите переход в `paid`/`delivered` при `total == 0` и запретите пустые корзины.

**Важно:** используйте только инструменты `pydantic` и стандартную библиотеку.


In [ ]:
from pydantic import BaseModel, EmailStr, field_validator, model_validator
from typing import List
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
from datetime import datetime
from enum import Enum

SKU_RE = re.compile(r"^[A-Z0-9]{3,12}$")

class OrderStatus(str, Enum):
    # TODO: перечислите статусы
    ...

class OrderItem(BaseModel):
    # TODO: опишите поля
    ...

    @field_validator("sku")
    @classmethod
    def sku_format(cls, v: str) -> str:
        ...

    @field_validator("qty")
    @classmethod
    def qty_positive(cls, v: int) -> int:
        ...

    @field_validator("unit_price")
    @classmethod
    def price_non_negative(cls, v: Decimal) -> Decimal:
        ...

class Order(BaseModel):
    # TODO: опишите поля
    ...

    @model_validator(mode="after")
    def check_business_rules(self):
        ...
        return self


## Задача 4. Конфигурация приложения (`BaseSettings`)

Опишите настройки подключения к внешнему API:

- `APISettings(BaseSettings)` с полями:
  - `base_url: HttpUrl`
  - `token: SecretStr`
  - `timeout_sec: int = 5` (1–60)
  - `retries: int = 2` (0–10)
- Используйте `model_config = ConfigDict(env_prefix="API_", env_file=".env", extra="ignore")`
- Проверьте, что значения корректно читаются из переменных окружения.

В тесте ниже среда заполняется вручную.


In [ ]:
import os
from pydantic_settings import BaseSettings
from pydantic import ConfigDict, SecretStr, HttpUrl, field_validator

class APISettings(BaseSettings):
    # TODO: поля и валидации
    ...

    # пример проверки диапазона для timeout_sec / retries
    @field_validator("timeout_sec", "retries")
    @classmethod
    def check_ranges(cls, v: int, info):
        ...

    model_config = ConfigDict(
        # TODO: настройте env_prefix и прочие опции
    )


## Задача 5. Извлечение из ORM (`from_attributes=True`)

Создайте простую SQLAlchemy-модель `SAUser(id, email, is_active)` (in-memory, без БД) и соответствующую модель Pydantic:

- Pydantic-модель `UserOut` с полями `id: UUID`, `email: EmailStr`, `is_active: bool`.
- Включите поддержку `from_attributes` в `model_config`.
- Создайте инстанс `SAUser` и провалидируйте его через `UserOut.model_validate(sa_user_instance)`.

Проверьте, что преобразование сработало.


In [ ]:
from typing import Optional
from sqlalchemy import Column, String, Boolean
from sqlalchemy.orm import declarative_base
from uuid import uuid4
from pydantic import BaseModel, EmailStr, ConfigDict

Base = declarative_base()

class SAUser(Base):
    __tablename__ = "users"
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    email = Column(String, nullable=False)
    is_active = Column(Boolean, default=True)

    def __init__(self, email: str, is_active: bool = True):
        self.id = str(uuid4())
        self.email = email
        self.is_active = is_active

class UserOut(BaseModel):
    # TODO: опишите поля и включите from_attributes
    ...
    model_config = ConfigDict(
        # TODO: включите режим атрибутов
    )


## Задача 6. JSON Schema и дружелюбные ошибки

1. Для модели из задачи 3 сгенерируйте JSON Schema (метод `model_json_schema`) и запишите его в переменную `ORDER_SCHEMA`.
2. Реализуйте функцию `safe_create_order(data: dict) -> tuple[bool, str]`, которая:
   - пытается создать `Order` из входного `dict`,
   - при успехе возвращает `(True, "<total=...>")`,
   - при ошибке возвращает `(False, "<короткое сообщение об ошибке>")` без стек-трейса.

Не используйте сторонние библиотеки.


In [ ]:
# Используем модели из задачи 3: OrderStatus, OrderItem, Order

ORDER_SCHEMA = ...  # TODO: сгенерируйте схему

def safe_create_order(data: dict) -> tuple[bool, str]:
    # TODO: реализуйте безопасное создание заказа
    ...
